In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # ✅ For saving scalers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# Load dataset
df = pd.read_csv("delhi_travel_data.csv")

# Features (input variables)
X = df[["Distance_KM", "Num_Signals", "Total_Signal_Wait", "Queue_Length",
        "Rush_Car", "Rush_Bike", "Rush_Cycle", "Rush_Bus", "Rush_Metro",
        "Pollution_Exposure_Car", "Pollution_Exposure_Bike", "Pollution_Exposure_Cycle",
        "Pollution_Exposure_Bus", "Pollution_Exposure_Metro"]]

# Targets (output variables)
y = df[["Time_Car", "Time_Bike", "Time_Cycle", "Time_Bus", "Time_Metro",
        "Expense_Car", "Expense_Bike", "Expense_Cycle", "Expense_Bus", "Expense_Metro",
        "Pollution_Car", "Pollution_Bike", "Pollution_Cycle", "Pollution_Bus", "Pollution_Metro"]]

# ✅ Normalize features & target values
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# ✅ Save the fitted scalers for later use in prediction
joblib.dump(scaler_X, "scaler_X.pkl")
joblib.dump(scaler_y, "scaler_y.pkl")

# Split data into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Build ANN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='linear')  # Output layer (multi-output regression)
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss=tf.keras.losses.MeanSquaredError(), metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, epochs=50, batch_size=15, validation_data=(X_test, y_test))

# Evaluate model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")
# Make Predictions
y_pred = model.predict(X_test)

# Compute R² Score
r2 = r2_score(y_test, y_pred)

print(f"R² Score: {r2:.4f}")  # Closer to 1 means better performance

# ✅ Save trained model
model.save("route_prediction_ann_v2.h5")

Epoch 1/50


C:\Users\adars\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


267/267 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1335 - mae: 0.2899 - val_loss: 0.0457 - val_mae: 0.1643
Epoch 2/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0382 - mae: 0.1459 - val_loss: 0.0313 - val_mae: 0.1256
Epoch 3/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0291 - mae: 0.1196 - val_loss: 0.0285 - val_mae: 0.1156
Epoch 4/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0269 - mae: 0.1119 - val_loss: 0.0275 - val_mae: 0.1123
Epoch 5/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0261 - mae: 0.1091 - val_loss: 0.0270 - val_mae: 0.1111
Epoch 6/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0257 - mae: 0.1082 - val_loss: 0.0268 - val_mae: 0.1108
Epoch 7/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0251 - mae: 0.1070 - val_loss: 0.0266 - val_mae: 0.1099
Epoch 8/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0252 - mae: 0.1067 - val_loss: 0.0265 - val_mae: 0.1095
Epoch 9/50
267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0249 

R² Score: 0.4834
